In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdate
import matplotlib.ticker as mtick
from datetime import datetime
import time

#df = pd.read_csv('datasets/2016-06-08_tests.csv')
df = pd.read_csv('datasets/2016-17-07_tests.csv')

f = lambda x: df[x] != 0
# Keep values from the accelerometer
df = df[f('acc_x') & f('acc_y') & f('acc_z')]

In [55]:
# some functions...

def extract_data_for(start_activity, end_activity, dataframe):
    data = dataframe[(dataframe['timestamp'] > start_activity) & (dataframe['timestamp'] < end_activity)]

    # Sort & reindex data
    data = data.sort_values(['timestamp'])
    data = data.reset_index(drop=True)
    return data

def format_time(date):
    count = 0
    datetime_ticks = []
    date_ticks = []
    while count < len(date):  
        if count % 100 == 0:
            datetime_ticks.append(date[count].item())
            date_ticks.append(time.strftime('%H:%M:%S', time.gmtime(date[count].item() / 1000)))

        count += 1
    return datetime_ticks, date_ticks

def plot_for(activity, data):
    date = data['timestamp']

    # Define a new x scale
    tick_val = format_time(date)[0]
    tick_lab = format_time(date)[1]

    plt.plot(date, data['acc_x'], color='red')
    plt.plot(date, data['acc_y'], color='green')
    plt.plot(date, data['acc_z'], color='blue')
    plt.xticks(tick_val, tick_lab)
    
    plt.xlabel('time')
    plt.ylabel('red: acc X, green: acc Y, blue: acc Z')
    plt.title('Acceleration over time for ' + activity)
    
    plt.show()
    
def mean(axis, data):
    return data[axis].mean()

def var(axis, data):
    return data[axis].var()

# 1 / n * ∑ √(x² + y² + z²)
def resultant(data):
    return np.sqrt(np.square(data).sum(axis=1)).sum() / len(data)

# 1 / n * √(∑ x_i²)
def norm_euc(axis, data):
    return np.sqrt(np.square(data[axis]).sum()) / len(data)

# 1 / n * ∑ |xi - mean_x|
def s_mean(axis, data):
    return np.abs(data[axis] - data[axis].mean()).sum() / len(data)

def minimum(data):
    min_x = np.min(data['acc_x'])
    min_y = np.min(data['acc_y'])
    min_z = np.min(data['acc_z'])
    return str(min_x) + ',' + str(min_y) + ',' + str(min_z)

def maximum(data):
    max_x = np.max(data['acc_x'])
    max_y = np.max(data['acc_y'])
    max_z = np.max(data['acc_z'])
    return str(max_x) + ',' + str(max_y) + ',' + str(max_z)

def amplitude(data):
    amplitude_x = np.max(data['acc_x']) - np.min(data['acc_x'])
    amplitude_y = np.max(data['acc_y']) - np.min(data['acc_y'])
    amplitude_z = np.max(data['acc_z']) - np.min(data['acc_z'])
    return str(amplitude_x) + ',' + str(amplitude_y) + ',' + str(amplitude_z)
    
def ovr(data):
    return np.sqrt(np.square(data['acc_x']) + np.square(data['acc_y']) + np.square(data['acc_z'])) - 9

In [63]:
# Graph for the 'straight line' activity
# len(sample) = 496

#start_straight_line = 1470487739978 
#end_straight_line = 1470487750096
#start_straight_line = 1470487779992 
#end_straight_line = 1470487790004
#start_straight_line = 1470487874986
#end_straight_line = 1470487885016

# dataset July
start_straight_line = 1468760000000
end_straight_line = 1468760085404
#start_straight_line = 1468760085404
#end_straight_line = 1468761050988

straight_line_data = extract_data_for(start_straight_line, end_straight_line, df)

activity = 'straight line'
#print(straight_line_data.head())
plot_for(activity, straight_line_data)

In [64]:
# min / max / amplitude
mini = minimum(straight_line_data)
maxi = maximum(straight_line_data)
amplitude = amplitude(straight_line_data)

# Compute the mean, for each axis
mean_acc_x = mean('acc_x', straight_line_data)
mean_acc_y = mean('acc_y', straight_line_data)
mean_acc_z = mean('acc_z', straight_line_data)

# Compute the variance, for each axis
var_acc_x = var('acc_x', straight_line_data)
var_acc_y = var('acc_y', straight_line_data)
var_acc_z = var('acc_z', straight_line_data)

# Compute the norm for each axis
norm_x = norm_euc('acc_x', straight_line_data)
norm_y = norm_euc('acc_y', straight_line_data)
norm_z = norm_euc('acc_z', straight_line_data)

# Compute s_mean, for each axis
s_mean_x = s_mean('acc_x', straight_line_data)
s_mean_y = s_mean('acc_y', straight_line_data)
s_mean_z = s_mean('acc_z', straight_line_data)

# Compute the resultant
resultant = resultant(straight_line_data)

# Compute ovr
ovr = ovr(straight_line_data)

# print the results
print(
    "min x,y,z : " + mini + "\n" +
    "max x,y,z : " + maxi + "\n" +
    "amplitude x,y,z : " + amplitude + "\n" +
    "mean x,y,z: " + str(mean_acc_x) + "," + str(mean_acc_y) + "," + str(mean_acc_z) + "\n" +
    "var x,y,z : " + str(var_acc_x) + "," + str(var_acc_y) + "," + str(var_acc_z) + "\n" +
    "norm x : " + str(norm_x) + "\n" +
    "norm y : " + str(norm_y) + "\n" + 
    "norm z : " + str(norm_z) + "\n" +
    "s_mean x : " + str(s_mean_x) + "\n" +
    "s_mean y : " + str(s_mean_y) + "\n" + 
    "s_mean z : " + str(s_mean_z) + "\n" +
    "resultant : " + str(resultant) + "\n" +
    "ovr : " + str(ovr)
)

min x,y,z : -0.25962126,-8.990676,4.123414
max x,y,z : 1.7887678,-6.604555,7.0322194
amplitude x,y,z : 2.04838906,2.386121,2.9088054
mean x,y,z: 0.447926397398,-7.81412498358,5.64516654577
var x,y,z : 0.107463914961,0.162080598835,0.195978338189
norm x : 0.0391175881415
norm y : 0.551893274557
norm z : 0.399395741669
s_mean x : 0.239024415524
s_mean y : 0.31005783441
s_mean z : 0.33274327531
resultant : 1.46876006611e+12
ovr : 0      0.017966
1      1.069398
2      0.421711
3     -0.058147
4      0.947073
5      0.507173
6      0.149594
7      0.721176
8      1.147529
9      0.561068
10     0.417019
11     0.375379
12     1.227811
13     0.583977
14     0.491752
15     0.898913
16     0.793123
17     0.278703
18     0.135417
19     0.272713
20     1.244092
21     1.185649
22     1.003862
23     0.881050
24     0.611847
25     0.392403
26     0.382951
27     0.685782
28     0.680338
29     0.726122
         ...   
171    0.690799
172    0.978336
173    0.855649
174    0.769960
175    0.

In [75]:
# Graph for the 'turn to the left' activity
# len(sample) = 492

start_turn_left = 1470488004999
end_turn_left = 1470488015030
#start_turn_left = 1470487984339 # perte de 3s au début sur l'enregistrement initial
#end_turn_left = 1470487991009

turn_left_data = extract_data_for(start_turn_left, end_turn_left, df)

activity = 'turn left'

plot_for(activity, turn_left_data)

In [23]:
# Graph for the 'roundabout' activity
# len(sample) = 835

start_roundabout = 1470487927986
end_roundabout = 1470487945001

roundabout_data = extract_data_for(start_roundabout, end_roundabout, df)

activity = 'roundabout'

plot_for(activity, roundabout_data)

In [9]:
# Graph for the 'normal acceleration' activity
# len(sample) = 204

start_n_acc = 1470488383991
end_n_acc = 1470488393015

n_acc_data = extract_data_for(start_n_acc, end_n_acc, df)

activity = 'normal acceleration'

plot_for(activity, n_acc_data)

In [10]:
# Graph for the 'normal braking' activity
# len(sample) = 243

start_n_braking = 1470488544993
end_n_braking = 1470488550003

n_braking_data = extract_data_for(start_n_braking, end_n_braking, df)

activity = 'normal braking'

plot_for(activity, n_braking_data)

In [34]:
# Graph for the 'left change' activity
# len(sample) = 393

start_l_change = 1470488823993
end_l_change = 1470488832034
#start_l_change = 1470488855992
#end_l_change = 1470488863032

l_change_data = extract_data_for(start_l_change, end_l_change, df)

activity = 'left change'

plot_for(activity, l_change_data)

In [11]:
# Graph for the 'right change' activity
# len(sample) = 329

start_r_change = 1470488943005
end_r_change = 1470488949706

r_change_data = extract_data_for(start_r_change, end_r_change, df)

activity = 'right change'

plot_for(activity, r_change_data)

In [12]:
# Graph for the 'U turn' activity
# len(sample) = 751

start_u_turn = 1470488090031
end_u_turn = 1470488105019

u_turn_data = extract_data_for(start_u_turn, end_u_turn, df)

activity = 'U turn'

plot_for(activity, u_turn_data)